# Using the Linux GPIO character device and LIBGPIOD

---
## Disclaimer:
Because:
- I am only using one dynamically linked file, AND
- I haven't created a separate disk for cross copiled linked libraries, AND
- quite frankly I have NOT yet found a sensible comprehensible useful human readable step by step description how to set up and use "pkg-config" with cross compilation..

I came up with a simplified way of doing this

---
## Install LIBGPIOD on the BBB3 target

NOTE: All of this is run on the BBB3 target device..

### Download the files
```
cd ~
mkdir builds
cd builds
sudo apt-get install -y libtool pkg-config autoconf-archive
sudo apt-get install -y build-essential git
git clone git://git.kernel.org/pub/scm/libs/libgpiod/libgpiod.git
```

### Compile the library and tools
There is a step in here to create an "m4" durectory. This prevents occasional autoconf errors.

Note that this does not build the python bindings
```
cd libgpiod
mkdir m4
./autogen.sh --enable-tools=yes --prefix=/usr/local
make
sudo make install
sudo ldconfig
```

### Test the installation
Test for the header file:
```
pkg-config --cflags libgpiod
```
This should return:
```
-I/usr/local/include
```

Test for the library:
```
pkg-config --lib libgpiod
```
This should return:
```
-L/usr/local/lib -lgpiod
```

---
## Set up the host environment

This is a **simple hack**. On the Linux host (i.e. the cross compile environment):
- cd to "/usr/local/lib"
- copy all the libgpiod files from the BBB3 target to the host. This lets the cross compiling work without needing to mess with pkg-config paths, i.e.
  - sudo scp debian@192.168.1.65:/usr/local/lib/libgpiod.* .
  
The rest is managed in the makefile and the BBB3 repository:
- there is a "sysinc" directory in BBB3 repository that has the headers of the system dynamic libraries
- the makefiles have the paths set up correctly, not using pkg-config